# Implementation of Softmax Regression from Scratch
:label:`sec_softmax_scratch`

Just as we implemented linear regression from scratch,
we believe that multiclass logistic (softmax) regression
is similarly fundamental and you ought to know
the gory details of how to implement it yourself.
As with linear regression, after doing things by hand
we will breeze through an implementation in DJL for comparison.
To begin, let us import the familiar packages.

In [1]:
%use @file[../djl.json]
// %load ../utils/djl-imports
// %load ../utils/plot-utils.ipynb
// %load ../utils/Training.java
// %load ../utils/FashionMnistUtils.java
// %load ../utils/ImageUtils.java
import ai.djl.basicdataset.cv.classification.FashionMnist
import ai.djl.metric.Metrics

// Saved in utils for future use
/* Sum a list of numbers over time */
class Accumulator(n: Int) {
    val data = FloatArray(n) { 0f }


    /* Adds a set of numbers to the array */
    fun add(args: FloatArray) {
        for (i in 0..args.size - 1) {
            data[i] += args[i]
        }
    }

    /* Resets the array */
    fun reset() {
        data.fill(0f)
    }

    /* Returns the data point at the given index */
    fun get(index: Int): Float {
        return data[index]
    }
}

fun getLong(nm: String, n: Long): Long {
    val name = System.getProperty(nm)
    return if (null == name) n.toLong() else name.toLong()
}

class Training {

    fun linreg(X: NDArray, w: NDArray, b: NDArray): NDArray {
        return X.dot(w).add(b);
    }

    fun squaredLoss(yHat: NDArray, y: NDArray): NDArray {
        return (yHat.sub(y.reshape(yHat.getShape())))
            .mul((yHat.sub(y.reshape(yHat.getShape()))))
            .div(2);
    }

    fun sgd(params: NDList, lr: Float, batchSize: Int) {
        for (param in params) {
            // Update param in place.
            // param = param - param.gradient * lr / batchSize
            param.subi(param.getGradient().mul(lr).div(batchSize));
        }
    }

    /**
     * Allows to do gradient calculations on a subManager. This is very useful when you are training
     * on a lot of epochs. This subManager could later be closed and all NDArrays generated from the
     * calculations in this function will be cleared from memory when subManager is closed. This is
     * always a great practice but the impact is most notable when there is lot of data on various
     * epochs.
     */
    fun sgd(params: NDList, lr: Float, batchSize: Int, subManager: NDManager) {
        for (param in params) {
            // Update param in place.
            // param = param - param.gradient * lr / batchSize
            val gradient = param.getGradient()
            gradient.attach(subManager);
            param.subi(gradient.mul(lr).div(batchSize))
        }
    }

    fun accuracy(yHat: NDArray, y: NDArray): Float {
        // Check size of 1st dimension greater than 1
        // to see if we have multiple samples
        if (yHat.getShape().size(1) > 1) {
            // Argmax gets index of maximum args for given axis 1
            // Convert yHat to same dataType as y (int32)
            // Sum up number of true entries
            return yHat.argMax(1)
                .toType(DataType.INT32, false)
                .eq(y.toType(DataType.INT32, false))
                .sum()
                .toType(DataType.FLOAT32, false)
                .getFloat();
        }
        return yHat.toType(DataType.INT32, false)
            .eq(y.toType(DataType.INT32, false))
            .sum()
            .toType(DataType.FLOAT32, false)
            .getFloat();
    }

    fun trainingChapter6(
        trainIter: ArrayDataset,
        testIter: ArrayDataset,
        numEpochs: Int,
        trainer: Trainer,
        evaluatorMetrics: MutableMap<String, DoubleArray>
    ): Double {

        trainer.setMetrics(Metrics())

        EasyTrain.fit(trainer, numEpochs, trainIter, testIter)

        val metrics = trainer.getMetrics()

        trainer.getEvaluators()
            .forEach { evaluator ->
                {
                    evaluatorMetrics.put(
                        "train_epoch_" + evaluator.getName(),
                        metrics.getMetric("train_epoch_" + evaluator.getName()).stream()
                            .mapToDouble { x -> x.getValue() }
                            .toArray())
                    evaluatorMetrics.put(
                        "validate_epoch_" + evaluator.getName(),
                        metrics
                            .getMetric("validate_epoch_" + evaluator.getName())
                            .stream()
                            .mapToDouble { x -> x.getValue() }
                            .toArray())
                }
            }

        return metrics.mean("epoch")
    }

    /* Softmax-regression-scratch */
    fun evaluateAccuracy(net: UnaryOperator<NDArray>, dataIterator: Iterable<Batch>): Float {
        val metric = Accumulator(2) // numCorrectedExamples, numExamples
        for (batch in dataIterator) {
            val X = batch.getData().head()
            val y = batch.getLabels().head()
            metric.add(floatArrayOf(accuracy(net.apply(X), y), y.size().toFloat()))
            batch.close()
        }
        return metric.get(0) / metric.get(1)
    }
    /* End Softmax-regression-scratch */

    /* MLP */
    /* Evaluate the loss of a model on the given dataset */
    fun evaluateLoss(
        net: UnaryOperator<NDArray>,
        dataIterator: Iterable<Batch>,
        loss: BinaryOperator<NDArray>
    ): Float {
        val metric = Accumulator(2) // sumLoss, numExamples

        for (batch in dataIterator) {
            val X = batch . getData ().head();
            val y = batch . getLabels ().head();
            metric.add(
                floatArrayOf(loss.apply(net.apply(X), y).sum().getFloat(), y.size().toFloat()) )
            batch.close()
        }
        return metric.get(0) / metric.get(1)
    }
    /* End MLP */
}


We will work with the Fashion-MNIST dataset, just introduced in :numref:`sec_fashion_mnist`,
setting up an iterator with batch size $256$. We also set it to randomly shuffled elements for each batch for the training set. 

In [2]:
val batchSize = 256
val randomShuffle = true

// get training and validation dataset
val trainingSet = FashionMnist.builder()
        .optUsage(Dataset.Usage.TRAIN)
        .setSampling(batchSize, randomShuffle)
        .optLimit(getLong("DATASET_LIMIT", Long.MAX_VALUE))
        .build()

val validationSet = FashionMnist.builder()
        .optUsage(Dataset.Usage.TEST)
        .setSampling(batchSize, false)
        .optLimit(getLong("DATASET_LIMIT", Long.MAX_VALUE))
        .build()

## Initializing Model Parameters

As in our linear regression example,
each example here will be represented by a fixed-length vector.
Each example in the raw data is a $28 \times 28$ image.
In this section, we will flatten each image,
treating them as $784$-long 1D vectors.
In the future, we will talk about more sophisticated strategies
for exploiting the spatial structure in images,
but for now we treat each pixel location as just another feature.

Recall that in softmax regression,
we have as many outputs as there are categories.
Because our dataset has $10$ categories,
our network will have an output dimension of $10$.
Consequently, our weights will constitute a $784 \times 10$ matrix
and the biases will constitute a $1 \times 10$ vector.
As with linear regression, we will initialize our weights $W$
with Gaussian noise and our biases to take the initial value $0$.

In [3]:
val numInputs = 784L
val numOutputs = 10L

val manager = NDManager.newBaseManager()
val W = manager.randomNormal(0f, 0.01f, Shape(numInputs, numOutputs), DataType.FLOAT32);
val b = manager.zeros(Shape(numOutputs), DataType.FLOAT32);
val params = NDList(W, b)

## The Softmax

Before implementing the softmax regression model,
let us briefly review how operators such as `sum()` work
along specific dimensions in an `NDArray`.
Given a matrix `X` we can sum over all elements (default) or only
over elements in the same axis, *i.e.*, the column (`new int[]{0}`) or the same row (`new int[]{1}`).
We wrap the axis in an int array since we can specify multiple axes as well.
For example if we call `sum()` with `new int[]{0, 1}`, it sums up the elements over both the rows and columns.
In this 2D array, this means the total sum of the elements within! 
Note that if `X` is an array with shape `($2$, $3$)`
and we sum over the columns (`X.sum(new int[]{0})`),
the result will be a (1D) vector with shape `($3$,)`.
If we want to keep the number of axes in the original array
(resulting in a 2D array with shape `($1$, $3$)`),
rather than collapsing out the dimension that we summed over
we can specify `true` when invoking `sum()`.

In [4]:
val X = manager.create(arrayOf(intArrayOf(1, 2, 3), intArrayOf(4, 5, 6)))
println(X)
println(X.sum(intArrayOf(0), true))
println(X.sum(intArrayOf(1), true))
println(X.sum(intArrayOf(0, 1), true))

ND: (2, 3) cpu() int32
[[ 1,  2,  3],
 [ 4,  5,  6],
]

ND: (1, 3) cpu() int32
[[ 5,  7,  9],
]

ND: (2, 1) cpu() int32
[[ 6],
 [15],
]

ND: (1, 1) cpu() int32
[[21],
]



We are now ready to implement the softmax function.
Recall that softmax consists of two steps:
First, we exponentiate each term (using `exp()`).
Then, we sum over each row (we have one row per example in the batch)
to get the normalization constants for each example.
Finally, we divide each row by its normalization constant,
ensuring that the result sums to $1$.
Before looking at the code, let us recall
how this looks expressed as an equation:

$$
\mathrm{softmax}(\mathbf{X})_{ij} = \frac{\exp(X_{ij})}{\sum_k \exp(X_{ik})}.
$$

The denominator, or normalization constant,
is also sometimes called the partition function
(and its logarithm is called the log-partition function).
The origins of that name are in [statistical physics](https://en.wikipedia.org/wiki/Partition_function_(statistical_mechanics))
where a related equation models the distribution
over an ensemble of particles.

In [5]:
fun softmax(X: NDArray): NDArray  {
    val Xexp = X.exp()
    val partition = Xexp.sum(intArrayOf(1), true)
    return Xexp.div(partition) // The broadcast mechanism is applied here
}

As you can see, for any random input,
we turn each element into a non-negative number.
Moreover, each row sums up to 1,
as is required for a probability.
Note that while this looks correct mathematically,
we were a bit sloppy in our implementation
because we failed to take precautions against numerical overflow or underflow
due to large (or very small) elements of the matrix,
as we did in :numref:`sec_naive_bayes`.

In [6]:
val X = manager.randomNormal(Shape(2, 5))
val Xprob = softmax(X)
println(Xprob)
println(Xprob.sum(intArrayOf(1)))

ND: (2, 5) cpu() float32
[[0.4817, 0.1687, 0.1143, 0.0855, 0.1497],
 [0.106 , 0.2568, 0.0916, 0.5211, 0.0244],
]

ND: (2) cpu() float32
[1., 1.]



## The Model

Now that we have defined the softmax operation,
we can implement the softmax regression model.
The below code defines the forward pass through the network.
Note that we flatten each original image in the batch
into a vector with length `numInputs` with the `reshape()` function
before passing the data through our model.

In [7]:
fun net(X: NDArray) : NDArray {
        val currentW = params.get(0);
        val currentB = params.get(1);
        return softmax(X.reshape(Shape(-1, numInputs)).dot(currentW).add(currentB))
}

## The Loss Function

Next, we need to implement the cross-entropy loss function,
introduced in :numref:`sec_softmax`.
This may be the most common loss function
in all of deep learning because, at the moment,
classification problems far outnumber regression problems.

Recall that cross-entropy takes the negative log likelihood
of the predicted probability assigned to the true label $-\log P(y \mid x)$.
Rather than iterating over the predictions with a Java `for` loop
(which tends to be inefficient),
we can use the NDArray `get()` function in conjunction with `NDIndex`
to let us easily select the appropriate terms
from the matrix of softmax entries. This is typically known as the `pick()`
operator in other frameworks such as `PyTorch`.
Below, we illustrate the usage on a toy example,
with $3$ categories and $2$ examples.

The `":, {}"` section of the `NDIndex` selects all arrays
and the `manager.create(new int[]{0, 2})` creates an
`NDArray` with the values 0 and 2 to pick the 0th and 2nd elements
for each respective `NDArray`.

Note: when using `NDIndex` in this way, the passed in `NDArray` used for picking
indices must be of type `int` or `long`. You can use the `toType()` function
to change the type of the `NDArray` which will be shown below.

In [8]:
val yHat = manager.create(arrayOf(floatArrayOf(0.1f, 0.3f, 0.6f), floatArrayOf(0.3f, 0.2f, 0.5f)))
//yHat.get(new NDIndex(":, {}", manager.create(new int[]{0, 2})));
//yHat.get(NDIndex(":, {}", manager.create(intArrayOf(0,2))))
val x = NDIndex(":,  {}", manager.create(intArrayOf(0,2)))
val y = NDIndex(":, 0:2")
yHat.get(y)

ND: (2, 2) cpu() float32
[[0.1, 0.3],
 [0.3, 0.2],
]


Now we can implement the cross-entropy loss function efficiently with just one line of code.

In [9]:
// Cross Entropy only cares about the target class's probability
// Get the column index for each row
class LossFunction {
    fun crossEntropy(yHat: NDArray, y: NDArray) : NDArray {
        // Here, y is not guranteed to be of datatype int or long
        // and in our case we know its a float32.
        // We must first convert it to int or long(here we choose int)
        // before we can use it with NDIndex to "pick" indices. 
        // It also takes in a boolean for returning a copy of the existing NDArray
        // but we don't want that so we pass in `false`.
        return yHat.get(NDIndex(":, {}", y.toType(DataType.INT32, false))).log().neg()
    }
}

## Classification Accuracy

Given the predicted probability distribution `yHat`,
we typically choose the class with highest predicted probability
whenever we must output a *hard* prediction.
Indeed, many applications require that we make a choice.
Gmail must categorize an email into Primary, Social, Updates, or Forums.
It might estimate probabilities internally,
but at the end of the day it has to choose one among the categories.

When predictions are consistent with the actual category `y`, they are correct.
The classification accuracy is the fraction of all predictions that are correct.
Although it can be difficult optimize accuracy directly (it is not differentiable),
it is often the performance metric that we care most about,
and we will nearly always report it when training classifiers.

To compute accuracy we do the following:
First, we execute `yHat.argMax(1)` where 1 is the axis
to gather the predicted classes
(given by the indices for the largest entries in each row).
The result has the same shape as the variable `y`.
Now we just need to check how frequently the two match.
Since the equality function `eq()` is datatype-sensitive
(e.g., a `float32` and a `float32` are never equal),
we also need to convert both to the same type (we pick `int32`).
The result is an `NDArray` containing entries of 0 (false) and 1 (true).
We then sum the number of true entries and convert the result to a float.
Finally, we get the mean by dividing by the number of data points.

In [10]:
// Saved in the utils for later use
fun accuracy(yHat: NDArray, y: NDArray) : Float {
    // Check size of 1st dimension greater than 1
    // to see if we have multiple samples
    if (yHat.getShape().size(1) > 1) {
        // Argmax gets index of maximum args for given axis 1
        // Convert yHat to same dataType as y (int32)
        // Sum up number of true entries
        return yHat.argMax(1).toType(DataType.INT32, false).eq(y.toType(DataType.INT32, false))
            .sum().toType(DataType.FLOAT32, false).getFloat()
    }
    return yHat.toType(DataType.INT32, false).eq(y.toType(DataType.INT32, false))
        .sum().toType(DataType.FLOAT32, false).getFloat()
}

We will continue to use the variables `yHat` and `y`
defined in the `pick()` function,
as the predicted probability distribution and label, respectively.
We can see that the first example's prediction category is $2$
(the largest element of the row is $0.6$ with an index of $2$),
which is inconsistent with the actual label, $0$.
The second example's prediction category is $2$
(the largest element of the row is $0.5$ with an index of $2$),
which is consistent with the actual label, $2$.
Therefore, the classification accuracy rate for these two examples is $0.5$.

In [11]:
val y = manager.create(intArrayOf(0,2))
accuracy(yHat, y) / y.size()

0.5

Similarly, we can evaluate the accuracy for model `net` on the dataset
(accessed via `dataIterator`).

In [12]:
import java.util.function.UnaryOperator
import java.util.function.BinaryOperator

Here `Accumulator` is a utility class to accumulate sums over multiple numbers.

In [13]:
// Saved in utils for future use        
/* Sum a list of numbers over time */
class Accumulator(n: Int) {
    val data = FloatArray(n) { 0f }
    
    
    /* Adds a set of numbers to the array */
    fun add(args: FloatArray) {
        for (i in 0..args.size-1) {
            data[i] += args[i]
        }
    }

    /* Resets the array */
    fun reset() {
        Arrays.fill(data, 0f)
    }

    /* Returns the data point at the given index */
    fun get(index: Int) : Float{
        return data[index]
    }
}

In [14]:
// Saved in the utils for future use
fun evaluateAccuracy( net: UnaryOperator<NDArray>,  dataIterator: Iterable<Batch>) : Float{
    val metric = Accumulator(2)  // numCorrectedExamples, numExamples
    val batch = dataIterator.iterator().next();
    val X = batch.getData().head();
    val y = batch.getLabels().head();
    metric.add(floatArrayOf(accuracy(net.apply(X), y), y.size().toFloat()))
    batch.close()

    return metric.get(0) / metric.get(1);
}

Because we initialized the `net` model with random weights,
the accuracy of this model should be close to random guessing,
i.e., $0.1$ for $10$ classes.

In [15]:
evaluateAccuracy(::net, validationSet.getData(manager))

0.11328125

## Model Training

The training loop for softmax regression should look strikingly familiar
if you read through our implementation
of linear regression in :numref:`sec_linear_scratch`.
Here we refactor the implementation to make it reusable.
First, we define a function to train for one data epoch.
Note that `updater()` is a general function to update the model parameters,
which accepts the batch size as an argument.
Currently, it is a wrapper of `Training.sgd()`.

In [16]:
interface ParamConsumer {
     fun accept(params: NDList, lr: Float, batchSize: Int)
}
val lr = 0.1f

fun trainEpochCh3(net: UnaryOperator<NDArray> , trainIter: Iterable<Batch> , loss: BinaryOperator<NDArray> , updater: ParamConsumer ) : FloatArray {
    val metric = Accumulator(3); // trainLossSum, trainAccSum, numExamples
    
    // Attach Gradients
    for (param in params) {
        param.setRequiresGradient(true)
    }
    
    for (batch in trainIter) {
        var X = batch.getData().head();
        val y = batch.getLabels().head();
        X = X.reshape(Shape(-1, numInputs));
            
        val gc = Engine.getInstance().newGradientCollector()
            // Minibatch loss in X and y
           val yHat = net.apply(X);
            val l = loss.apply(yHat, y);
            gc.backward(l);  // Compute gradient on l with respect to w and b
            metric.add(floatArrayOf(l.sum().toType(DataType.FLOAT32, false).getFloat(), 
                                   accuracy(yHat, y), 
                                   y.size().toFloat()))
            gc.close()
        
        updater.accept(params, lr, batch.getSize());  // Update parameters using their gradient
        
        batch.close();
    }
    // Return trainLoss, trainAccuracy
    return floatArrayOf(metric.get(0) / metric.get(2), metric.get(1) / metric.get(2))
}

Before showing the implementation of the training function, we define a utility class that draws data in animation. Again, it aims to simplify the code in later chapters.

In [17]:
/*
import tech.tablesaw.api.Row;
import tech.tablesaw.columns.Column;

// Saved in utils
/* Animates a graph with real-time data. */
class Animator {
    private String id; // Id reference of graph(for updating graph)
    private Table data; // Data Points
    
    public Animator() {
        id = "";
        
        // Incrementally plot data
        data = Table.create("Data")
        .addColumns(
            FloatColumn.create("epoch", new float[]{}),
            FloatColumn.create("value", new float[]{}),
            StringColumn.create("metric", new String[]{})
        );
    }

    // Add a single metric to the table
    public void add(float epoch, float value, String metric) {
        Row newRow = data.appendRow();
        newRow.setFloat("epoch", epoch);
        newRow.setFloat("value", value);
        newRow.setString("metric", metric);
    }
    
    // Add accuracy, train accuracy, and train loss metrics for a given epoch
    // Then plot it on the graph
    public void add(float epoch, float accuracy, float trainAcc, float trainLoss) {
        add(epoch, trainLoss, "train loss");
        add(epoch, trainAcc, "train accuracy");
        add(epoch, accuracy, "test accuracy");
        show();
    }
    
    // Display the graph
    public void show() {
        if (id.equals("")) {
            id = display(LinePlot.create("", data, "epoch", "value", "metric"));
            return;
        }
        update();
    }
    
    // Update the graph
    public void update() {
        updateDisplay(id, LinePlot.create("", data, "epoch", "value", "metric"));
    }
    
    // Returns the column at the given index
    // if it is a float column
    // Otherwise returns null
    public float[] getY(Table t, int index) {
        Column c = t.column(index);
        if (c.type() == ColumnType.FLOAT) {
            float[] newArray = new float[c.size()];
            System.arraycopy(c.asList().toArray(), 0, newArray, 0, c.size());
            return newArray;
        }
        return null;
    }
}
*/

The training function then runs multiple epochs and visualize the training progress.

In [18]:
fun trainCh3(net: UnaryOperator<NDArray> , trainDataset: Dataset , testDataset:Dataset , 
                     loss: BinaryOperator<NDArray> , numEpochs: Int, updater: ParamConsumer ) 
                                                             {
//    Animator animator = new Animator();
    for (i in 1..numEpochs) {
        val trainMetrics = trainEpochCh3(net, trainDataset.getData(manager), loss, updater);
        val accuracy = evaluateAccuracy(net, testDataset.getData(manager));
        val trainAccuracy = trainMetrics[1];
        val trainLoss = trainMetrics[0];
        
//        animator.add(i, accuracy, trainAccuracy, trainLoss);
        println("Epoch %d: Test Accuracy: %f".format(i, accuracy))
        println("Train Accuracy: %f".format(trainAccuracy))
        println("Train Loss: %f".format(trainLoss))
    }
}

Again, we use the minibatch stochastic gradient descent
to optimize the loss function of the model.
Note that the number of epochs (`numEpochs`),
and learning rate (`lr`) are both adjustable hyper-parameters.
By changing their values, we may be able
to increase the classification accuracy of the model.
In practice we will want to split our data three ways
into training, validation, and test data,
using the validation data to choose
the best values of our hyper-parameters.

In [19]:
val numEpochs = 5;
val lr = 0.1f;

class Updater {
    fun updater(params:NDList , lr: Float, batchSize: Int) {
        Training().sgd(params, lr, batchSize);
    }
}

trainCh3(::net, trainingSet, validationSet, LossFunction::crossEntropy, numEpochs, Updater::updater);

Line_19.jupyter-kts (10:45 - 71) Type mismatch: inferred type is KFunction3<Line_9.LossFunction, NDArray, NDArray, NDArray> but (NDArray, NDArray) -> NDArray was expected

## Prediction

Now that training is complete,
our model is ready to classify some images.
Given a series of images,
we will compare their actual labels
(first line of text output)
and the model predictions
(second line of text output).

In [20]:
// Saved in the FashionMnistUtils class for later use
// Number should be < batchSize for this function to work properly
public BufferedImage predictCh3(UnaryOperator<NDArray> net, ArrayDataset dataset, int number, NDManager manager) 
    throws IOException, TranslateException {
    int[] predLabels = new int[number];
    
    Batch batch = dataset.getData(manager).iterator().next();
    NDArray X = batch.getData().head();
    int[] yHat = net.apply(X).argMax(1).toType(DataType.INT32, false).toIntArray();
    for (int i = 0; i < number; i++) {
        predLabels[i] = yHat[i];
    }
    
    return FashionMnistUtils.showImages(dataset, predLabels, 28, 28, 4, manager);
}

predictCh3(Net::net, validationSet, 6, manager)

Line_20.jupyter-kts (3:59 - 59) Expecting an element
Line_20.jupyter-kts (3:81 - 81) Expecting an element
Line_20.jupyter-kts (3:93 - 93) Expecting an element
Line_20.jupyter-kts (3:112 - 112) Expecting an element
Line_20.jupyter-kts (4:23 - 23) Expecting an element
Line_20.jupyter-kts (4:23 - 24) Expecting an element
Line_20.jupyter-kts (5:9 - 9) Expecting an index element
Line_20.jupyter-kts (5:22 - 23) Expecting an element
Line_20.jupyter-kts (7:17 - 18) Expecting an element
Line_20.jupyter-kts (7:26 - 27) Expecting an element
Line_20.jupyter-kts (8:15 - 16) Expecting an element
Line_20.jupyter-kts (8:22 - 23) Expecting an element
Line_20.jupyter-kts (8:31 - 31) Expecting an expression
Line_20.jupyter-kts (9:9 - 9) Expecting an index element
Line_20.jupyter-kts (9:16 - 17) Expecting an element
Line_20.jupyter-kts (9:21 - 22) Expecting an element
Line_20.jupyter-kts (10:14 - 15) Expecting 'in'
Line_20.jupyter-kts (10:15 - 15) Expecting ')'
Line_20.jupyter-kts (10:16 - 16) Expecting a

In [ ]:
import ai.djl.basicdataset.cv.classification.FashionMnist
import ai.djl.engine.Engine
import ai.djl.metric.Metrics
import ai.djl.ndarray.NDArray
import ai.djl.ndarray.NDList
import ai.djl.ndarray.NDManager
import ai.djl.ndarray.index.NDIndex
import ai.djl.ndarray.types.DataType
import ai.djl.ndarray.types.Shape
import ai.djl.training.EasyTrain
import ai.djl.training.Trainer
import ai.djl.training.dataset.ArrayDataset
import ai.djl.training.dataset.Batch
import ai.djl.training.dataset.Dataset
import java.util.function.BinaryOperator
import java.util.function.UnaryOperator

class Test2 {
}

// Saved in utils for future use
/* Sum a list of numbers over time */
class Accumulator(n: Int) {
    val data = FloatArray(n) { 0f }


    /* Adds a set of numbers to the array */
    fun add(args: FloatArray) {
        for (i in 0..args.size - 1) {
            data[i] += args[i]
        }
    }

    /* Resets the array */
    fun reset() {
        data.fill(0f)
    }

    /* Returns the data point at the given index */
    fun get(index: Int): Float {
        return data[index]
    }
}

fun getLong(nm: String, n: Long): Long {
    val name = System.getProperty(nm)
    return if (null == name) n.toLong() else name.toLong()
}

class Training {

    fun linreg(X: NDArray, w: NDArray, b: NDArray): NDArray {
        return X.dot(w).add(b);
    }

    fun squaredLoss(yHat: NDArray, y: NDArray): NDArray {
        return (yHat.sub(y.reshape(yHat.getShape())))
            .mul((yHat.sub(y.reshape(yHat.getShape()))))
            .div(2);
    }

    fun sgd(params: NDList, lr: Float, batchSize: Int) {
        for (param in params) {
            // Update param in place.
            // param = param - param.gradient * lr / batchSize
            param.subi(param.getGradient().mul(lr).div(batchSize));
        }
    }

    /**
     * Allows to do gradient calculations on a subManager. This is very useful when you are training
     * on a lot of epochs. This subManager could later be closed and all NDArrays generated from the
     * calculations in this function will be cleared from memory when subManager is closed. This is
     * always a great practice but the impact is most notable when there is lot of data on various
     * epochs.
     */
    fun sgd(params: NDList, lr: Float, batchSize: Int, subManager: NDManager) {
        for (param in params) {
            // Update param in place.
            // param = param - param.gradient * lr / batchSize
            val gradient = param.getGradient()
            gradient.attach(subManager);
            param.subi(gradient.mul(lr).div(batchSize))
        }
    }

    fun accuracy(yHat: NDArray, y: NDArray): Float {
        // Check size of 1st dimension greater than 1
        // to see if we have multiple samples
        if (yHat.getShape().size(1) > 1) {
            // Argmax gets index of maximum args for given axis 1
            // Convert yHat to same dataType as y (int32)
            // Sum up number of true entries
            return yHat.argMax(1)
                .toType(DataType.INT32, false)
                .eq(y.toType(DataType.INT32, false))
                .sum()
                .toType(DataType.FLOAT32, false)
                .getFloat();
        }
        return yHat.toType(DataType.INT32, false)
            .eq(y.toType(DataType.INT32, false))
            .sum()
            .toType(DataType.FLOAT32, false)
            .getFloat();
    }

    fun trainingChapter6(
        trainIter: ArrayDataset,
        testIter: ArrayDataset,
        numEpochs: Int,
        trainer: Trainer,
        evaluatorMetrics: MutableMap<String, DoubleArray>
    ): Double {

        trainer.setMetrics(Metrics())

        EasyTrain.fit(trainer, numEpochs, trainIter, testIter)

        val metrics = trainer.getMetrics()

        trainer.getEvaluators()
            .forEach { evaluator ->
                {
                    evaluatorMetrics.put(
                        "train_epoch_" + evaluator.getName(),
                        metrics.getMetric("train_epoch_" + evaluator.getName()).stream()
                            .mapToDouble { x -> x.getValue() }
                            .toArray())
                    evaluatorMetrics.put(
                        "validate_epoch_" + evaluator.getName(),
                        metrics
                            .getMetric("validate_epoch_" + evaluator.getName())
                            .stream()
                            .mapToDouble { x -> x.getValue() }
                            .toArray())
                }
            }

        return metrics.mean("epoch")
    }

    /* Softmax-regression-scratch */
    fun evaluateAccuracy(net: UnaryOperator<NDArray>, dataIterator: Iterable<Batch>): Float {
        val metric = Accumulator(2) // numCorrectedExamples, numExamples
        for (batch in dataIterator) {
            val X = batch.getData().head()
            val y = batch.getLabels().head()
            metric.add(floatArrayOf(accuracy(net.apply(X), y), y.size().toFloat()))
            batch.close()
        }
        return metric.get(0) / metric.get(1)
    }
    /* End Softmax-regression-scratch */

    /* MLP */
    /* Evaluate the loss of a model on the given dataset */
    fun evaluateLoss(
        net: UnaryOperator<NDArray>,
        dataIterator: Iterable<Batch>,
        loss: BinaryOperator<NDArray>
    ): Float {
        val metric = Accumulator(2) // sumLoss, numExamples

        for (batch in dataIterator) {
            val X = batch . getData ().head();
            val y = batch . getLabels ().head();
            metric.add(
                floatArrayOf(loss.apply(net.apply(X), y).sum().getFloat(), y.size().toFloat()) )
            batch.close()
        }
        return metric.get(0) / metric.get(1)
    }
    /* End MLP */
}

val batchSize = 256
val randomShuffle = true

// get training and validation dataset
val trainingSet = FashionMnist.builder()
    .optUsage(Dataset.Usage.TRAIN)
    .setSampling(batchSize, randomShuffle)
    .optLimit(getLong("DATASET_LIMIT", Long.MAX_VALUE))
    .build()

val validationSet = FashionMnist.builder()
    .optUsage(Dataset.Usage.TEST)
    .setSampling(batchSize, false)
    .optLimit(getLong("DATASET_LIMIT", Long.MAX_VALUE))
    .build()

val numInputs = 784L
val numOutputs = 10L

val manager = NDManager.newBaseManager()
val W = manager.randomNormal(0f, 0.01f, Shape(numInputs, numOutputs), DataType.FLOAT32);
val b = manager.zeros(Shape(numOutputs), DataType.FLOAT32);
val params = NDList(W, b)

val X = manager.create(arrayOf(intArrayOf(1, 2, 3), intArrayOf(4, 5, 6)))

fun softmax(X: NDArray): NDArray  {
    val Xexp = X.exp()
    val partition = Xexp.sum(intArrayOf(1), true)
    return Xexp.div(partition) // The broadcast mechanism is applied here
}

val X0 = manager.randomNormal(Shape(2, 5))
val Xprob = softmax(X0)

fun net(X: NDArray) : NDArray {
    val currentW = params.get(0);
    val currentB = params.get(1);
    return softmax(X.reshape(Shape(-1, numInputs)).dot(currentW).add(currentB))
}

val yHat = manager.create(arrayOf(floatArrayOf(0.1f, 0.3f, 0.6f), floatArrayOf(0.3f, 0.2f, 0.5f)))
//yHat.get(new NDIndex(":, {}", manager.create(new int[]{0, 2})));
//yHat.get(NDIndex(":, {}", manager.create(intArrayOf(0,2))))
val x = NDIndex(":,  {}", manager.create(intArrayOf(0,2)))
val y = NDIndex(":, 0::2")

// Cross Entropy only cares about the target class's probability
// Get the column index for each row
//class LossFunction {
    fun crossEntropy(yHat: NDArray, y: NDArray) : NDArray {
        // Here, y is not guranteed to be of datatype int or long
        // and in our case we know its a float32.
        // We must first convert it to int or long(here we choose int)
        // before we can use it with NDIndex to "pick" indices.
        // It also takes in a boolean for returning a copy of the existing NDArray
        // but we don't want that so we pass in `false`.
        return yHat.get(NDIndex(":, {}", y.toType(DataType.INT32, false))).log().neg()
    }
//}

// Saved in the utils for later use
fun accuracy(yHat: NDArray, y: NDArray) : Float {
    // Check size of 1st dimension greater than 1
    // to see if we have multiple samples
    if (yHat.getShape().size(1) > 1) {
        // Argmax gets index of maximum args for given axis 1
        // Convert yHat to same dataType as y (int32)
        // Sum up number of true entries
        return yHat.argMax(1).toType(DataType.INT32, false).eq(y.toType(DataType.INT32, false))
            .sum().toType(DataType.FLOAT32, false).getFloat()
    }
    return yHat.toType(DataType.INT32, false).eq(y.toType(DataType.INT32, false))
        .sum().toType(DataType.FLOAT32, false).getFloat()
}

val y0 = manager.create(intArrayOf(0,2))

// Saved in the utils for future use
fun evaluateAccuracy( net: UnaryOperator<NDArray>,  dataIterator: Iterable<Batch>) : Float{
    val metric = Accumulator(2)  // numCorrectedExamples, numExamples
    val batch = dataIterator.iterator().next();
    val X = batch.getData().head();
    val y = batch.getLabels().head();
    metric.add(floatArrayOf(accuracy(net.apply(X), y), y.size().toFloat()))
    batch.close()

    return metric.get(0) / metric.get(1);
}

interface ParamConsumer {
    fun accept(params: NDList, lr: Float, batchSize: Int)
}
val lr = 0.1f

fun trainEpochCh3(net: UnaryOperator<NDArray>,
                  trainIter: Iterable<Batch>,
                  loss: BinaryOperator<NDArray>,
                  updater: (NDList, Float, Int)->Unit ) : FloatArray {
    val metric = Accumulator(3); // trainLossSum, trainAccSum, numExamples

    // Attach Gradients
    for (param in params) {
        param.setRequiresGradient(true)
    }

    for (batch in trainIter) {
        var X = batch.getData().head();
        val y = batch.getLabels().head();
        X = X.reshape(Shape(-1, numInputs));

        val gc = Engine.getInstance().newGradientCollector()
        // Minibatch loss in X and y
        val yHat = net.apply(X);
        val l = loss.apply(yHat, y);
        gc.backward(l);  // Compute gradient on l with respect to w and b
        metric.add(floatArrayOf(l.sum().toType(DataType.FLOAT32, false).getFloat(),
            accuracy(yHat, y),
            y.size().toFloat()))
        gc.close()

        updater(params, lr, batch.getSize());  // Update parameters using their gradient

        batch.close();
    }
    // Return trainLoss, trainAccuracy
    return floatArrayOf(metric.get(0) / metric.get(2), metric.get(1) / metric.get(2))
}

fun trainCh3(net: UnaryOperator<NDArray> , trainDataset: Dataset , testDataset:Dataset ,
             loss: BinaryOperator<NDArray> , numEpochs: Int, updater: (NDList, Float, Int)->Unit )
{
//    Animator animator = new Animator();
    for (i in 1..numEpochs) {
        val trainMetrics = trainEpochCh3(net, trainDataset.getData(manager), loss, updater);
        val accuracy = evaluateAccuracy(net, testDataset.getData(manager));
        val trainAccuracy = trainMetrics[1];
        val trainLoss = trainMetrics[0];

//        animator.add(i, accuracy, trainAccuracy, trainLoss);
        println("Epoch %d: Test Accuracy: %f".format(i, accuracy))
        println("Train Accuracy: %f".format(trainAccuracy))
        println("Train Loss: %f".format(trainLoss))
    }
}

val numEpochs = 5;
val lr0 = 0.1f;

//class Updater {
    fun updater(params:NDList , lr: Float, batchSize: Int) {
        Training().sgd(params, lr, batchSize)
    }
//}


fun main() {
    println(X)
    println(X.sum(intArrayOf(0), true))
    println(X.sum(intArrayOf(1), true))
    println(X.sum(intArrayOf(0, 1), true))

    println(Xprob)
    println(Xprob.sum(intArrayOf(1)))
    println(yHat.get(x))
    println(yHat.get(y))
    println(accuracy(yHat, y0) / y0.size())
    println(evaluateAccuracy(::net, validationSet.getData(manager)))
    val xx = trainCh3(::net, trainingSet, validationSet, ::crossEntropy, numEpochs, ::updater)
    println(xx)
}

## Summary

With softmax regression, we can train models for multi-category classification.
The training loop is very similar to that in linear regression:
retrieve and read data, define models and loss functions,
then train models using optimization algorithms.
As you will soon find out, most common deep learning models
have similar training procedures.

## Exercises

1. In this section, we directly implemented the softmax function based on the mathematical definition of the softmax operation. What problems might this cause (hint: try to calculate the size of $\exp(50)$)?
1. The function `crossEntropy()` in this section is implemented according to the definition of the cross-entropy loss function.  What could be the problem with this implementation (hint: consider the domain of the logarithm)?
1. What solutions you can think of to fix the two problems above?
1. Is it always a good idea to return the most likely label. E.g., would you do this for medical diagnosis?
1. Assume that we want to use softmax regression to predict the next word based on some features. What are some problems that might arise from a large vocabulary?